# Flux Examples
From https://comfyanonymous.github.io/ComfyUI_examples/flux/

In [ ]:
from comfy_script.runtime import *

# load('http://127.0.0.1:8188/')
load()

# Nodes can only be imported after load()
from comfy_script.runtime.nodes import *

## Flux Dev

In [ ]:
with Workflow():
    model, clip, vae = CheckpointLoaderSimple('flux1-dev-fp8.safetensors')

    positive_conditioning = CLIPTextEncode('cute anime girl with massive fluffy fennec ears and a big fluffy tail blonde messy long hair blue eyes wearing a maid outfit with a long black gold leaf pattern dress and a white apron mouth open placing a fancy black forest cake with candles on top of a dinner table of an old dark Victorian mansion lit by candlelight with a bright window to the foggy forest and very expensive stuff everywhere there are paintings on the walls', clip)
    positive_conditioning = FluxGuidance(positive_conditioning, 3.5)
    negative_conditioning = CLIPTextEncode('', clip)

    latent = EmptySD3LatentImage(1024, 1024, 1)
    # Note that Flux dev and schnell do not have any negative prompt so CFG should be set to 1.0. Setting CFG to 1.0 means the negative prompt is ignored.
    latent = KSampler(model, 972054013131368, 20, 1, 'euler', 'simple', positive_conditioning, negative_conditioning, latent, 1)
    image = VAEDecode(latent, vae)
    SaveImage(image, 'ComfyUI')

### FP8 Checkpoint version

In [ ]:
with Workflow():
    '''If you get an error in any of the nodes above make sure the files are in the correct directories.

    See the top of the examples page for the links : https://comfyanonymous.github.io/ComfyUI_examples/flux/

    flux1-dev.safetensors goes in: ComfyUI/models/unet/

    t5xxl_fp16.safetensors and clip_l.safetensors go in: ComfyUI/models/clip/

    ae.safetensors goes in: ComfyUI/models/vae/


    Tip: You can set the weight_dtype above to one of the fp8 types if you have memory issues.'''

    noise = RandomNoise(219670278747233)

    width = 1024
    height = 1024

    model = UNETLoader('flux1-dev.safetensors', 'default')
    # The reference sampling implementation auto adjusts the shift value based on the resolution, if you don't want this you can just comment out (Ctrl+/) this ModelSamplingFlux node.
    model = ModelSamplingFlux(model, 1.15, 0.5, width, height)

    clip = DualCLIPLoader('t5xxl_fp16.safetensors', 'clip_l.safetensors', 'flux')
    positive_conditioning = CLIPTextEncode('cute anime girl with massive fluffy fennec ears and a big fluffy tail blonde messy long hair blue eyes wearing a maid outfit with a long black gold leaf pattern dress and a white apron mouth open holding a fancy black forest cake with candles on top in the kitchen of an old dark Victorian mansion lit by candlelight with a bright window to the foggy forest and very expensive stuff everywhere', clip)
    positive_conditioning = FluxGuidance(positive_conditioning, 3.5)

    guider = BasicGuider(model, positive_conditioning)
    sampler = KSamplerSelect('euler')
    sigmas = BasicScheduler(model, 'simple', 20, 1)
    latent = EmptySD3LatentImage(width, height, 1)
    latent, _ = SamplerCustomAdvanced(noise, guider, sampler, sigmas, latent)
    vae = VAELoader('ae.safetensors')
    image = VAEDecode(latent, vae)
    SaveImage(image, 'ComfyUI')

## Redux
The redux model lets you prompt with images. It can be used with any Flux1 dev or schnell model workflow.

In [ ]:
with Workflow():
    '''If you get an error in any of the nodes above make sure the files are in the correct directories.

    See the top of the examples page for the links : https://comfyanonymous.github.io/ComfyUI_examples/flux/

    flux1-dev.safetensors goes in: ComfyUI/models/unet/

    t5xxl_fp16.safetensors and clip_l.safetensors go in: ComfyUI/models/clip/

    ae.safetensors goes in: ComfyUI/models/vae/


    Tip: You can set the weight_dtype above to one of the fp8 types if you have memory issues.'''

    noise = RandomNoise(958831004022715)

    width = 1024
    height = 1024

    model = UNETLoader('flux1-dev.safetensors', 'default')
    # The reference sampling implementation auto adjusts the shift value based on the resolution, if you don't want this you can just comment out (Ctrl+/) this ModelSamplingFlux node.
    model = ModelSamplingFlux(model, 1.15, 0.5, width, height)

    clip = DualCLIPLoader('t5xxl_fp16.safetensors', 'clip_l.safetensors', 'flux')
    positive_conditioning = CLIPTextEncode('cute anime girl with massive fluffy fennec ears', clip)
    positive_conditioning = FluxGuidance(positive_conditioning, 3.5)

    style_model = StyleModelLoader('flux1-redux-dev.safetensors')
    clip_vision = CLIPVisionLoader('sigclip_vision_patch14_384.safetensors')
    # You can chain multiple `StyleModelApply` nodes if you want to mix multiple images together.
    image, _ = LoadImage('sd3_controlnet_example.png')
    clip_vision_output = CLIPVisionEncode(clip_vision, image)
    positive_conditioning = StyleModelApply(positive_conditioning, style_model, clip_vision_output)

    guider = BasicGuider(model, positive_conditioning)
    sampler = KSamplerSelect('euler')
    sigmas = BasicScheduler(model, 'simple', 20, 1)
    latent = EmptySD3LatentImage(width, height, 1)
    latent, _ = SamplerCustomAdvanced(noise, guider, sampler, sigmas, latent)
    vae = VAELoader('ae.safetensors')
    image = VAEDecode(latent, vae)
    SaveImage(image, 'ComfyUI')